In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

llm_config = {"model": "gpt-4o"}

In [2]:
task = """
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       """

In [3]:
import autogen
import tqdm as notebook_tqdm

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
    "blogpost (with title) on given topics. You must polish your "
    "writing based on the feedback you receive and give a refined "
    "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

### Unleashing Potential with DeepLearning.AI

DeepLearning.AI is revolutionizing the world of artificial intelligence by democratizing access to high-quality, hands-on education. Founded by AI visionary Andrew Ng, the platform offers diverse, industry-relevant courses that transform novices into skilled practitioners. Learners benefit from cutting-edge curriculum, real-world projects, and a community of supportive peers and mentors. Whether you're aiming to supercharge your career or innovate within your current role, DeepLearning.AI provides the knowledge and tools to turn your AI aspirations into tangible achievements. Embark on your AI journey today and discover endless possibilities with DeepLearning.AI.


In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
    "the writer and provide constructive "
    "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Unleashing Potential with DeepLearning.AI

DeepLearning.AI is revolutionizing the world of artificial intelligence by democratizing access to high-quality, hands-on education. Founded by AI visionary Andrew Ng, the platform offers diverse, industry-relevant courses that transform novices into skilled practitioners. Learners benefit from cutting-edge curriculum, real-world projects, and a community of supportive peers and mentors. Whether you're aiming to supercharge your career or innovate within your current role, DeepLearning.AI provides the knowledge and tools to turn your AI aspirations into tangible achievements. Embark on your AI journey today and discover endless possibilities with DeepLearning.AI.

--------------------

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
    "your ability to optimize content for search engines, "
    "ensuring that it ranks well and attracts organic traffic. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
    "your ability to ensure that content is legally compliant "
    "and free from any potential legal issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
    "your ability to ensure that content is ethically sound "
    "and free from any potential ethical issues. "
    "Make sure your suggestion is concise (within 3 bullet points), "
    "concrete and to the point. "
    "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f"""Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"""


review_chats = [
    {
        "recipient": SEO_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",
        },
        "max_turns": 1,
    },
    {
        "recipient": legal_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'Reviewer': '', 'Review': ''}.",
        },
        "max_turns": 1,
    },
    {
        "recipient": ethics_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return review into as JSON object only:"
            "{'reviewer': '', 'review': ''}",
        },
        "max_turns": 1,
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1,
    },
]

In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Unleashing Potential with DeepLearning.AI

DeepLearning.AI is revolutionizing the world of artificial intelligence by democratizing access to high-quality, hands-on education. Founded by AI visionary Andrew Ng, the platform offers diverse, industry-relevant courses that transform novices into skilled practitioners. Learners benefit from cutting-edge curriculum, real-world projects, and a community of supportive peers and mentors. Whether you're aiming to supercharge your career or innovate within your current role, DeepLearning.AI provides the knowledge and tools to turn your AI aspirations into tangible achievements. Embark on your AI journey today and discover endless possibilities with DeepLearning.AI.

--------------------

In [17]:
print(res.summary)

### Unleashing Potential with DeepLearning.AI

Discover unparalleled AI education with DeepLearning.AI, founded by Andrew Ng. Offering premier AI courses and artificial intelligence training, this platform transforms novices into skilled practitioners. Benefit from a cutting-edge curriculum, hands-on projects, and a supportive community. While outcomes may vary, DeepLearning.AI equips you with advanced skills to significantly enhance your career prospects. Embark on your AI journey today. Explore our [AI courses](course-link) and see where AI can take you with DeepLearning.AI.
